- 都是 policy gradient，都是用 gradient descent 来优化
    - 使得有利的“动作”概率上升，不利的概率下降：
    - 交叉熵损失通过梯度下降机制，增大正确类别的概率，同时减少错误类别的概率，符合分类任务的优化目标。
    - 交叉熵损失（监督学习）：
        - 目标：使模型预测的概率分布逼近真实标签的 one-hot 分布。
        - 更新逻辑：正确类别的概率上升，错误类别的概率下降。
    - 策略梯度（强化学习）：
        - 目标：最大化未来累积奖励的期望。
        - 更新逻辑：若某个动作带来高奖励，则提高其概率；反之则降低。

### 监督学习

In [1]:
import torch
import torch.nn.functional as F

# 初始化模型参数（logits），使得 softmax 后得到初始概率 [0.2, 0.5, 0.3]
logits = torch.log(torch.tensor([0.2, 0.5, 0.3]))  # 将概率转换为 logits
logits.requires_grad_(True)  # 启用梯度追踪

target = torch.tensor(0)  # 真实标签是 "北京"（索引0）

# 计算交叉熵损失（自动包含 softmax）
loss = F.cross_entropy(logits.unsqueeze(0), target.unsqueeze(0))  # 添加 batch 维度

# 反向传播计算梯度
loss.backward()

print("原始梯度:", logits.grad)  # 梯度公式：∂loss/∂logits = (softmax_output - one_hot_target)

# 梯度下降更新（学习率设为 0.1）
with torch.no_grad():
    logits -= 0.1 * logits.grad  # 执行参数更新

# 计算新的概率分布
new_probs = F.softmax(logits, dim=0)

print("更新后的概率分布:", new_probs.detach().numpy().round(4))

原始梯度: tensor([-0.8000,  0.5000,  0.3000])
更新后的概率分布: [0.2203 0.4836 0.296 ]


In [2]:
logits.grad

tensor([-0.8000,  0.5000,  0.3000])

## RL policy gradient

In [7]:
# 初始化策略网络参数
logits = torch.tensor([0.2, 0.5, 0.3]).log().requires_grad_(True)
action_probs = F.softmax(logits, dim=0)

# 采样动作并获取回报（假设动作 0 的回报 G=1.0）
sampled_action = 0
reward = 1.0

# 计算策略梯度
obj = torch.log(action_probs[sampled_action]) * reward
obj.backward()

# 更新 logits（高回报动作的概率上升）
with torch.no_grad():
    logits += 0.1 * logits.grad

# 新的概率分布
new_probs = F.softmax(logits, dim=0)

In [8]:
new_probs

tensor([0.2203, 0.4836, 0.2960], grad_fn=<SoftmaxBackward0>)